# Cantera Tutorial: Python

## Getting Started

Let's get started with Cantera by importing the Cantera and NumPy libraries. We can also print the version of Cantera that we're using with the `__version__` attribute from the `cantera` module, typically aliased as `ct`.

In [1]:
import cantera as ct
import numpy as np

print(f"Using Cantera version {ct.__version__}")

Using Cantera version 2.6.0


When using Cantera, the first thing you usually need is an object representing some phase of matter. Here, we'll create a gas mixture:

In [2]:
gas1 = ct.Solution('gri30.yaml')

To view the state of the mixture, *call* the `gas1` object as if it were a function:

In [3]:
gas1()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   0.081894 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy             26469             53361  J
   internal energy       -1.2108e+06        -2.441e+06  J
           entropy             64910        1.3086e+05  J/K
    Gibbs function       -1.9447e+07       -3.9204e+07  J
 heat capacity c_p             14311             28851  J/K
 heat capacity c_v             10187             20536  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -15.717
     [  +52 minor]                 0                 0  



What you have just done is created an object `gas1` that implements GRI-Mech 3.0, the 53-species, 325-reaction natural gas combustion mechanism developed by Gregory P. Smith, David M. Golden, Michael Frenklach, Nigel W. Moriarty, Boris Eiteneer, Mikhail Goldenberg, C. Thomas Bowman, Ronald K. Hanson, Soonho Song, William C. Gardiner, Jr., Vitali V. Lissianski, and Zhiwei Qin.  See the [GRI-Mech Home Page](http://combustion.berkeley.edu/gri-mech/) for more information.

The `gas1` object has properties you would expect for a gas mixture: a temperature, a pressure, species mole and mass fractions, etc. As we will soon see, it has many more properties.

The summary of the state of `gas1` that you found above shows that the new objects created from the `gri30.cti` input file start out with a temperature of 300 K, a pressure of 1 atm, and have a composition that consists of only one species, in this case hydrogen. There is nothing special about H2—it just happens to be the first species listed in the input file defining GRI-Mech 3.0. In general, whichever species is listed first will initially have a mole fraction of 1.0, and all others will be zero.

## Setting the State

The state of the object can easily be changed. For example:

In [4]:
gas1.TP = 1200, 101325

sets the temperature to 1200 K and the pressure to 101325 Pa (Cantera always uses SI units + kmol). After this statement, calling `gas1()` results in:

In [5]:
gas1()


  gri30:

       temperature   1200 K
          pressure   1.0133e+05 Pa
           density   0.020473 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        1.3295e+07        2.6802e+07  J
   internal energy        8.3457e+06        1.6825e+07  J
           entropy             85222        1.7181e+05  J/K
    Gibbs function       -8.8972e+07       -1.7937e+08  J
 heat capacity c_p             15377             31000  J/K
 heat capacity c_v             11253             22686  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -17.978
     [  +52 minor]                 0                 0  



Thermodynamics generally requires that *two* properties in addition to composition information be specified to fix the intensive state of a substance (or mixture). The state of the mixture can be set using several combinations of two properties. The following are all equivalent:

In [6]:
gas1.TP = 1200, 101325            # temperature, pressure
gas1.TD = 1200, 0.0204723         # temperature, density
gas1.HP = 1.32956e7, 101325       # specific enthalpy, pressure
gas1.UV = 8.34619e6, 1/0.0204723  # specific internal energy, specific volume
gas1.SP = 85227.6, 101325         # specific entropy, pressure
gas1.SV = 85227.6, 1/0.0204723    # specific entropy, specific volume

Cantera can set and get properties on a molar basis (J/kmol) or a mass basis (J/kg). Note that the mass basis is set by default, so all the values in the previous cell are per unit mass. The basis of a `Solution` instance can be changed by assigning to the `basis` attribute of the instance:

In [7]:
gas1.basis = 'molar'
gas1.basis = 'mass'

Properties may be also **read** independently, such as

In [8]:
gas1.T

1200.5188172713504

or

In [9]:
gas1.h

13302755.250164837

or together:

In [10]:
gas1.UV

(8351530.632807602, 48.84649013545132)

In [12]:
water = ct.PureFluid('liquidvapor.yaml', 'water')
water.TP = 1200, 101325
water.TPX = None, None, 1.0

The composition can be set in terms of either mole fractions (`X`) or mass fractions (`Y`) by assigning to the corresponding attribute of the `Solution` instance. There are three main options to set the composition of a mixture:

* A string specifying the species names and relative mole numbers

      "CH4:1, O2:2, N2:7.52"
      
* A Python dictionary where the keys are species names and the values are relative mole numbers

      {"CH4": 1, "O2": 2, "N2": 7.52}

* A NumPy array of length `n_species`

In any of these case, the mole numbers are normalized so the sum is 1.0.

In [13]:
gas1.X = "CH4:0.8, O2:2, N2:7.52"
print(gas1.mole_fraction_dict())

{'CH4': 0.07751937984496124, 'N2': 0.7286821705426355, 'O2': 0.19379844961240308}


In [14]:
phi = 0.8
gas1.X = {'CH4':1, 'O2':2/phi, 'N2': 2*3.76/phi}
print(gas1.mole_fraction_dict())

{'CH4': 0.07751937984496125, 'N2': 0.7286821705426356, 'O2': 0.19379844961240314}


In [15]:
nsp = gas1.n_species
gas1.Y = np.ones(nsp)
gas1()


  gri30:

       temperature   1200.5 K
          pressure   13437 Pa
           density   0.020472 kg/m^3
  mean mol. weight   15.208 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        1.2528e+07        1.9053e+08  J
   internal energy        1.1872e+07        1.8055e+08  J
           entropy             17372         2.642e+05  J/K
    Gibbs function       -8.3274e+06       -1.2665e+08  J
 heat capacity c_p            2898.6             44084  J/K
 heat capacity c_v            2351.9             35769  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2          0.018868           0.14234           -21.949
                 H          0.018868           0.28467            3.1625
                 O          0.018868          0.017935           -2.0944


One additional method is available to set the equivalence ratio directly, called [`set_equivalence_ratio()`](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/thermo.html#cantera.ThermoPhase.set_equivalence_ratio). In this case, it is assumed that all C atoms are oxidized to CO2, H atoms to H2O, and S to SO2. Other atoms are assumed not to react (e.g., N ends up as N2). The signature for this method is:

    set_equivalence_ratio(phi, fuel, oxidizer)
    
where the `phi` argument is a number that represents the desired equivalence ratio of the mixture and the `fuel` and `oxidizer` represent the fuel and oxidizer mixtures in any of the formats shown before on a molar basis. For instance, to set the equivalence ratio to 0.8 with an equimolar fuel mixture of methane and propane and an oxidizer of air, the code is:

In [16]:
gas1.set_equivalence_ratio(phi, {"CH4": 1, "C3H8": 1}, "O2:1, N2:3.76")
print(gas1.mole_fraction_dict())

{'C3H8': 0.022909507445589918, 'CH4': 0.02290950744558992, 'N2': 0.7537227949599085, 'O2': 0.2004581901489118}


When the composition alone is changed, the **temperature** and **density** are held constant. This means that the pressure and other intensive properties will change. The composition can also be set in conjunction with the intensive properties of the mixture:

In [17]:
gas1.TPX = 1200, 101325, "CH4:1, O2:2, N2:7.52"
gas1()


  gri30:

       temperature   1200 K
          pressure   1.0133e+05 Pa
           density   0.28063 kg/m^3
  mean mol. weight   27.633 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        8.6193e+05        2.3818e+07  J
   internal energy        5.0087e+05        1.3841e+07  J
           entropy            8914.2        2.4633e+05  J/K
    Gibbs function       -9.8351e+06       -2.7178e+08  J
 heat capacity c_p            1397.3             38611  J/K
 heat capacity c_v            1096.4             30296  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.22014           0.19011           -28.747
               CH4          0.055187          0.095057           -35.961
                N2           0.72467           0.71483           -25.67

When setting the state, you can control what properties are held constant by passing the special value `None` to the property setter. For example, to change the specific volume to 2.1 m<sup>3</sup>/kg while holding entropy constant:

In [18]:
gas1.SV = None, 2.1
gas1()


  gri30:

       temperature   1383.5 K
          pressure   1.9823e+05 Pa
           density   0.47619 kg/m^3
  mean mol. weight   27.633 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        1.1224e+06        3.1017e+07  J
   internal energy        7.0614e+05        1.9513e+07  J
           entropy            8914.2        2.4633e+05  J/K
    Gibbs function       -1.1211e+07       -3.0979e+08  J
 heat capacity c_p            1439.9             39788  J/K
 heat capacity c_v              1139             31474  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.22014           0.19011           -28.513
               CH4          0.055187          0.095057           -35.115
                N2           0.72467           0.71483           -25.

Or to set the mass fractions while holding temperature and pressure constant:

In [19]:
gas1.TPY = None, None, "CH4:1.0, O2:0.5"

## Working with a Subset of Species

In [20]:
print(gas1.species())

[<Species H2>, <Species H>, <Species O>, <Species O2>, <Species OH>, <Species H2O>, <Species HO2>, <Species H2O2>, <Species C>, <Species CH>, <Species CH2>, <Species CH2(S)>, <Species CH3>, <Species CH4>, <Species CO>, <Species CO2>, <Species HCO>, <Species CH2O>, <Species CH2OH>, <Species CH3O>, <Species CH3OH>, <Species C2H>, <Species C2H2>, <Species C2H3>, <Species C2H4>, <Species C2H5>, <Species C2H6>, <Species HCCO>, <Species CH2CO>, <Species HCCOH>, <Species N>, <Species NH>, <Species NH2>, <Species NH3>, <Species NNH>, <Species NO>, <Species NO2>, <Species N2O>, <Species HNO>, <Species CN>, <Species HCN>, <Species H2CN>, <Species HCNN>, <Species HCNO>, <Species HOCN>, <Species HNCO>, <Species NCO>, <Species N2>, <Species AR>, <Species C3H7>, <Species C3H8>, <Species CH2CHO>, <Species CH3CHO>]


Many properties of a [`Solution`](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/importing.html#cantera.Solution) provide values for each species present in the phase. If you want to get values only for a subset of these species, you can use Python's "slicing" syntax to select data for just the species of interest. To get the mole fractions of just the major species in `gas1`, in the order specified, you can write:

In [21]:
Xmajor = gas1['CH4','O2','CO2','H2O','N2'].X
print(Xmajor)

[0.79956021 0.20043979 0.         0.         0.        ]


If you want to use the same set of species repeatedly, you can keep a reference to the sliced phase object:

In [22]:
major = gas1['CH4','O2','CO2','H2O','N2']
cp_major = major.partial_molar_cp
wdot_major = major.net_production_rates
print(wdot_major)

[-0.00599455 -0.00500799  0.          0.          0.        ]


The slice object and the original object share the same internal state, so modifications to one will affect the other.

In [23]:
gas1.TPX = 1200, 101325, "CH4:1, N2:7.52, O2:2"
print(major.net_production_rates)
print(major.X)

[-8.65079074e-06 -8.53258691e-06  0.00000000e+00  0.00000000e+00
 -3.28511288e-13]
[0.09505703 0.19011407 0.         0.         0.7148289 ]


## Working with Mechanism Files

In the previous example, we created an object that models an ideal gas mixture with the species and reactions of GRI-Mech 3.0, using the `gri30.cti` input file included with Cantera. This is a CTI input file and is relatively easy for humans to read and write. Cantera also supports an XML-based input file format that is easy for Cantera to parse, but hard for humans to write. Several reaction mechanism files in both formats are included with Cantera, including ones that model high-temperature air, a hydrogen/oxygen reaction mechanism, and a few surface reaction mechanisms. These files are usually located in the `data` subdirectory of the Cantera installation directory, e.g., `C:\Program Files\Cantera\data` on Windows or `/usr/local/cantera/data/` on Unix/Linux/Mac OS X machines, depending on how you installed Cantera and the options you specified.

There are a number of mechanism files included with Cantera, including the `gri30.cti` example we saw earlier.

In [24]:
from pathlib import Path
p = Path(ct.__file__)
print([c.name for c in (p.parent / "data").glob("*.yaml")])

['ohn.yaml', 'nDodecane_Reitz.yaml', 'graphite.yaml', 'h2o2.yaml', 'water.yaml', 'KOH.yaml', 'nasa_gas.yaml', 'gri30.yaml', 'methane_pox_on_pt.yaml', 'silicon_carbide.yaml', 'nasa_condensed.yaml', 'silicon.yaml', 'airNASA9.yaml', 'liquidvapor.yaml', 'critical-properties.yaml', 'sofc.yaml', 'diamond.yaml', 'lithium_ion_battery.yaml', 'air.yaml', 'element-standard-entropies.yaml', 'gri30_ion.yaml', 'ptcombust.yaml', 'silane.yaml', 'gri30_highT.yaml']


Cantera input files are plain text files, and can be created with any text editor. See the document *[Defining Phases](https://cantera.org/tutorials/cti/defining-phases.html)* for more information.

A Cantera input file may contain more than one phase specification, or may contain specifications of interfaces (surfaces). Here, we import definitions of two bulk phases and the interface between them from the file `diamond.cti`:

In [25]:
gas2 = ct.Solution('diamond.yaml', 'gas')
diamond = ct.Solution('diamond.yaml', 'diamond')
diamond_surf = ct.Interface('diamond.yaml', 'diamond_100', [gas2, diamond])

Note that the bulk (i.e., 3D or homogenous) phases that participate in the surface reactions must also be passed as arguments to [`Interface`](http://cantera.github.io/docs/sphinx/html/cython/importing.html#cantera.Interface).

### Converting CK-format files

Cantera also comes with a script to convert CHEMKIN (CK)-format input files to the CTI format. We'll cover that in the [`chemkin_conversion.ipynb`](chemkin_conversion.ipynb) Notebook.

## Getting Help

In addition to the Sphinx-generated *[Python Module Documentation](https://cantera.org/documentation/docs-2.4/sphinx/html/index.html)*, documentation of the Python classes and their methods can be accessed from within the Python interpreter as well.

Suppose you have created a Cantera object and want to know what methads are avialable for it, and get help on using the methods:

In [26]:
g = ct.Solution("gri30.yaml")

To get help on the Python class that this object is an instance of, put a question mark `?` after the variable:

In [ ]:
g?

For a simple list of the properties and methods of this object:

In [28]:
dir(g)

['CK_mode',
 'DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'Te',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__composition_to_array',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_cython__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__setstate_cython__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_cinit',
 '_enable_plasma',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_init_yaml',
 '_native_state',
 '_partial_states',
 '_references',
 'activities',
 'act

To get help on a specific method, e.g. the `species_index` method:

In [ ]:
g.species_index?

For properties, getting the documentation is slightly trickier, as the usual method will give you help for the *result*, e.g.:

In [ ]:
g.T?

provides help on Python's `float` class. To get the help for the temperature property, ask for the attribute of the class object itself:

In [27]:
g.__class__.T?

Help can also be obtained using the `help` function:

In [28]:
help(g.species_index)

Help on built-in function species_index:

species_index(...) method of cantera.composite.Solution instance
    ThermoPhase.species_index(self, species) -> int
    
    The index of species *species*, which may be specified as a string or
    an integer. In the latter case, the index is checked for validity and
    returned. If no such species is present, an exception is thrown.



## Chemical Equilibrium

To set a gas mixture to a state of chemical equilibrium, use the `equilibrate` method:

In [31]:
g = ct.Solution("gri30.yaml")
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("TP")
g()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   1.1248 kg/m^3
  mean mol. weight   27.689 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.8723e+06       -7.9532e+07  J
   internal energy       -2.9624e+06       -8.2026e+07  J
           entropy            7226.6         2.001e+05  J/K
    Gibbs function       -5.0403e+06       -1.3956e+08  J
 heat capacity c_p            1106.5             30638  J/K
 heat capacity c_v            806.22             22323  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2          0.011038         0.0095511           -29.325
               H2O           0.11807           0.18147           -121.37
               CO2           0.14422          0.090735           -185.87


The above statement sets the state of object `g` to the state of chemical equilibrium holding temperature and pressure fixed. Alternatively, the specific enthalpy and pressure can be held fixed:

In [32]:
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("HP")
g()


  gri30:

       temperature   2188.9 K
          pressure   1.0133e+05 Pa
           density   0.15349 kg/m^3
  mean mol. weight   27.57 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.4244e+05        -6.684e+06  J
   internal energy       -9.0257e+05       -2.4884e+07  J
           entropy            9804.1         2.703e+05  J/K
    Gibbs function       -2.1703e+07       -5.9836e+08  J
 heat capacity c_p            1500.5             41370  J/K
 heat capacity c_v            1198.9             33055  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2        0.00013397         0.0018322           -26.072
                 H        8.2844e-06        0.00022659           -13.036
                 O        0.00015045        0.00025925           -16.8

Other options are:
* `'UV'` for fixed specific internal energy and specific volume
* `'SV'` for fixed specific entropy and specific volume
* `'SP'` for fixed specific entropy and pressure

How can you tell if `equilibrate` has correctly found the chemical equilibrium state? One way is to verify that the net rates of progress of all reversible reactions are zero. Here is the code to do this:

In [33]:
g.TPX = 300.0, ct.one_atm, 'CH4:0.95, O2:2, N2:7.52'
g.equilibrate('HP')

In [34]:
rf = g.forward_rates_of_progress
rr = g.reverse_rates_of_progress
for i in range(g.n_reactions):
    if g.is_reversible(i) and rf[i] != 0.0:
        print(f"{i:4d}\t{(rf[i] - rr[i])/rf[i]:10.4g}")

   0	-1.762e-16
   1	-3.702e-15
   2	-7.872e-16
   3	 5.418e-15
   4	 -5.49e-15
   5	 1.253e-16
   6	 3.538e-15
   7	         0
   8	 3.218e-15
   9	-3.649e-15
  10	 3.438e-15
  11	 1.351e-16
  12	-3.656e-15
  13	-3.441e-15
  14	  3.22e-15
  15	 1.387e-15
  16	-1.743e-16
  17	-7.062e-16
  18	 7.666e-15
  19	         0
  20	 7.291e-15
  21	 7.106e-16
  22	 3.912e-15
  23	-3.804e-15
  24	 2.055e-15
  25	-3.544e-15
  26	-1.311e-14
  27	-3.702e-15
  28	 5.243e-16
  29	-3.464e-15
  30	-1.052e-15
  31	-7.965e-15
  32	-6.967e-15
  33	-6.939e-15
  34	-6.885e-15
  35	-6.839e-15
  37	-2.091e-15
  38	-1.423e-16
  39	-2.286e-16
  40	-2.692e-16
  41	-1.518e-16
  42	-3.422e-15
  43	 1.927e-15
  44	 3.537e-15
  45	 3.679e-15
  46	-1.136e-15
  47	 -1.06e-14
  48	 3.367e-15
  49	 3.859e-15
  50	-1.841e-15
  51	-5.674e-15
  52	 9.625e-16
  53	-6.697e-15
  54	-7.039e-15
  55	-3.611e-15
  56	 -1.13e-15
  57	 7.633e-15
  58	-7.351e-15
  59	 5.358e-15
  60	 5.014e-15
  61	 1.113e-15
  62	-1.065e-14
  63	-1.

/var/folders/ng/d9rd9fb92c7bxz9fy7vwpj7m0000gp/T/ipykernel_96509/1859125551.py:4: DeprecationWarning: 'is_reversible' is deprecated and will be removed after Cantera 2.6.
Replaceable by property 'reversible' of the corresponding reaction object.
  if g.is_reversible(i) and rf[i] != 0.0:


If the magnitudes of the numbers in this list are all very small (which in this case they are), then each reversible reaction is very nearly equilibrated, which only occurs if the gas is in chemical equilibrium.

You might be wondering how `equilibrate` works. (Then again, you might not.) Method `equilibrate` invokes Cantera's chemical equilibrium solver, which uses an element potential method. The element potential method is one of a class of equivalent *nonstoichiometric* methods that all have the characteristic that the probelm reduces to solving a set of $M$ nonlinear algebraic equations, where $M$ is the number of elements (not species). The so-called *stoichiometric* methods, on the other hand (including the Gibbs minimization), require solving $K$ nonlinear equations, where $K$ is the number of species (usually $K >> M$). See Smith and Missen's "Chemical Reaction Equilibrium Analysis" for more information on the various algorithms and their characteristics.

Cantera uses a damped Newton method to solve these equations, and does a few other things to generate a good starting guess and to produce a reasonably robust algorithm. If you want to know more about the details, look at the on-line documentated source code of Cantera C++ class [`ChemEquil.h`](https://cantera.org/documentation/docs-2.4/doxygen/html/d4/dd4/ChemEquil_8h.html).